In [2]:
from music21 import *
import os 

In [3]:
def open_midi(midi_path):
    mf = converter.parse(midi_path)
    return mf

In [5]:
mf = open_midi('C:/Users/PC/Desktop/Music Generation/dataset/1_WorldRunner.mid')

In [6]:
# Play midi file
mf.show("midi")

In [7]:
# mf.show() # Display sheet music in musescore

In [8]:
# Check if there are notes which should have ended before given offset
def checkForNoteOffEvent(currentOffset, noteOffEvents):
    notesToEnd = []
    
    for noteOffEvent in noteOffEvents: # for (notename, endingOffset)
        if noteOffEvent[1] <= currentOffset:
            notesToEnd.append(noteOffEvent)
            
    return notesToEnd

In [9]:
# Restricts possible velocities to 8 values, keeping the number of unique note events smaller
# Resembles ppp, pp, p, mp, mf, f, ff, fff dynamics 
def vModifier(velocity):
    if (velocity == 0):
        return 0
    
    velocity = min(127, ((velocity // 16) + 1) * 16)
    return velocity

def tModifier(tempo):
    if (tempo == 0):
        return 0
    
    tempo = ((tempo // 10) + 1) * 10
    return tempo

In [10]:
# Access midifile with Parts merged together with correct offsets

def midi2text(midifile):
    previousElementOffset = 0.0
    offsetChanged = False

    tempoRetrieved = False
    timeSigRetrieved = False

    tokens = []
    noteOffEvents = []

    tokens.append("START")

    for element in midifile.flat.elements:
        #print(type(element))

        currentElementOffset = element.offset

        notesToEnd = checkForNoteOffEvent(currentElementOffset, noteOffEvents)

        if (len(notesToEnd) != 0):
            for noteToEnd in notesToEnd:
                if (noteToEnd[1] - previousElementOffset > 0):
                    tokens.append("wait:" + str(round(float(noteToEnd[1] - previousElementOffset), 3)))
                    previousElementOffset = noteToEnd[1]
                tokens.append("note:" + str(noteToEnd[0]) + ":OFF")
                noteOffEvents.remove(noteToEnd)

        # If offset has increased and we're looking at new notes, add a wait event before adding the new notes
        if (currentElementOffset != previousElementOffset and (isinstance(element, note.Note) or isinstance(element, chord.Chord))):
            offsetChanged = True
            tokens.append("wait:" + str(round(float(currentElementOffset - previousElementOffset), 3)))

        if (isinstance(element, tempo.MetronomeMark) and not tempoRetrieved):
            tempoRetrieved = True
            tokens.append("tempo:" + str(tModifier(element.number)))

        if (isinstance(element, meter.TimeSignature) and not timeSigRetrieved):
            timeSigRetrieved = True
            tokens.append("timesig:" + str(element.ratioString))

        if (isinstance(element, note.Note)): # This is a note event, add a token for this note
            tokens.append("note:" + str(element.pitch) + ":v" + str(vModifier(element.volume.velocity)))
            noteOffEvents.append((str(element.pitch), round(float(currentElementOffset + element.duration.quarterLength), 3)))

        if (isinstance(element, chord.Chord)): # This is a chord event, add a token for each note in chord
            for chordnote in element: 
                tokens.append("note:" + str(chordnote.pitch) + ":v" + str(vModifier(element.volume.velocity)))
                noteOffEvents.append((str(chordnote.pitch), round(float(currentElementOffset + element.duration.quarterLength), 3)))

        if (offsetChanged):
            previousElementOffset = currentElementOffset
            offsetChanged = False

    # Finally make sure that all notes that end after the offset of the last element of mf.flat.elements are given an off event.
    for noteToEnd in noteOffEvents.copy():
        if (previousElementOffset - noteToEnd[1] != 0):
            tokens.append("wait:" + str(round(float(noteToEnd[1] - currentElementOffset), 3)))
            previousElementOffset = noteToEnd[1]
        tokens.append("note:" + str(noteToEnd[0]) + ":OFF")
        noteOffEvents.remove(noteToEnd)
        
    if (len(noteOffEvents) != 0):
        print("Not all notes have note-off events")

    tokens.append("END")
    return tokens

midiTokens = midi2text(mf)

In [11]:
def text2midi(tokens):
    s = stream.Stream()

    currentOffset = 0
    currentToken = 0

    for token in tokens:

        splitToken = token.split(":")

        if token.startswith("tempo"):
            s.append(tempo.MetronomeMark(number=float(splitToken[1])))

        if token.startswith("timesig"):
            s.append(meter.TimeSignature(splitToken[1]))

        if token.startswith("note") and not token.endswith("OFF"):
            noteDuration = 0
            noteName = splitToken[1]
            noteVelocity = int(splitToken[2][1:])

            for element in tokens[currentToken+1:]:
                splitToken2 = element.split(":")
                if (element.startswith("wait")):
                    noteDuration += float(splitToken2[1])
                if (element.startswith("note") and element.endswith("OFF")):
                    if (noteName == splitToken2[1]):
                        newNote = note.Note(nameWithOctave=splitToken[1],  
                               quarterLength=float(noteDuration))
                        newNote.volume.velocity = int(splitToken[2][1:])
                        s.insert(currentOffset, newNote)
                        break

        if token.startswith("wait"):
            currentOffset += float(splitToken[1]) 

        currentToken += 1

    return s

text2midi(midiTokens).show("midi")

In [13]:
path = '../dataset/'
filenames = os.listdir(path)

with open("../dataset_text/miditokens.txt", "a") as f:
    progress = 0
    for filename in filenames:
        midifile = open_midi(path + filename)
        f.write(' '.join(midi2text(midifile)) + '\n')
        progress += 1
        
        if (progress % 20 == 0):
            print('.', end='')

....................................